In [ ]:
!pip install networkx

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import networkx as nx

import pandas as pd

import copy
import random

import networkx as nx
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch_geometric.nn import GCNConv
from torch_geometric.data import Batch, Data, Dataset

from torch.optim.lr_scheduler import ReduceLROnPlateau

from sklearn.preprocessing import MinMaxScaler
import time

import math


# Data Prep

In [ ]:
files = ["donken",
         "Holmgrens",
         "IONITY",
         "Jureskogs_Vattenfall",
         "UFC"]

nr_of_data_points = 163618
splits=[0.8, 0.9]

data_dict = {}


for f in files:
    data = pd.read_csv('data/varnamo/data_' + f + '_5T_k-10.csv')
    #data.info()
    data.set_index('Unnamed: 0', inplace=True)
    
    data = data.drop(columns=data.columns.difference(['Occupancy']))
    
    data_dict[f] = data


# Model Making

In [ ]:
# Create a graph
G = nx.Graph()

# Add nodes
num_nodes = 5
G.add_nodes_from(range(num_nodes))

Jureskogs_Vattenfall = 0
IONITY = 1
donken = 2
Holmgrens = 3
UFC = 4

# Define edges to connect specific nodes with custom weights
edges_to_connect = [
    (Jureskogs_Vattenfall, IONITY, 230),
    (Jureskogs_Vattenfall, UFC, 750),
    (Jureskogs_Vattenfall, Holmgrens, 750),
    (Jureskogs_Vattenfall, donken, 650),
    (IONITY, UFC, 550),
    (IONITY, Holmgrens, 500),
    (IONITY, donken, 450),
    (UFC, Holmgrens, 280),
    (UFC, donken, 550),
    (Holmgrens, donken, 550)
]

# Add edges with custom weights
for edge in edges_to_connect:
    G.add_edge(edge[0], edge[1], weight=edge[2])

# Create adjacency matrix with weights
adj_matrix = nx.adjacency_matrix(G).todense()

torch_adj_matrix = torch.Tensor(adj_matrix)

edge_index = torch_adj_matrix.nonzero(as_tuple=False).t().contiguous()
edge_attr = torch_adj_matrix[torch_adj_matrix.nonzero()].reshape(-1, 1)


# Define colors for nodes
node_colors = ['Yellow'] * num_nodes

print(G.nodes)

# Draw the graph
pos = nx.spring_layout(G)  # positions for all nodes
nx.draw(G, pos, with_labels=True, node_color=node_colors, node_size=700, font_size=10)
edge_labels = nx.get_edge_attributes(G, 'weight')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)

# Show the graph
plt.show()


In [ ]:
future_steps = 36
seq_len = 576
batch_size = 64


indices = list(range(len(data) - future_steps - seq_len))
print(len(indices))
random.shuffle(indices)
print(len(indices))

train_i = indices[:int(len(indices)*0.8)] 
val_i = indices[int(len(indices)*0.8):int(len(indices)*0.9)]
test_i = indices[int(len(indices)*0.9):]


class datasetMaker(Dataset):
    def __init__(self, station_data, indices_conversion, edge_index, edge_attr, seq_len, future_steps, batch_size):
        self.station_data = station_data
        self.indices_conversion = indices_conversion
        self.size = station_data["donken"].shape[0]
        self.edge_index = edge_index
        self.edge_attr = edge_attr
        self.seq_len = seq_len
        self.future_steps = future_steps
        self.batch_size = batch_size

    def __len__(self):
        return len(self.indices_conversion) - self.seq_len - self.future_steps

    def __getitem__(self, index):
        
        index = self.indices_conversion[index]
        
        seq_end = index + self.seq_len
        fut_end = index + self.seq_len + self.future_steps
        
        node_features = []
        for i, (station, data) in enumerate(self.station_data.items()):
            node_feature = data.iloc[index:seq_end].values
            node_features.append(node_feature)
        node_features = torch.tensor(np.array(node_features)).float()

        labels = []
        for i, (station, data) in enumerate(self.station_data.items()):
            label = data.iloc[seq_end:fut_end].values
            labels.append(label)
        labels = torch.unsqueeze(torch.tensor(np.array(labels)), dim=2).float()
        
        Gdata = Data(x=node_features, y=labels, edge_index=self.edge_index, edge_attr=self.edge_attr)

        return Gdata, labels
    
def custom_collate(batch):
    label = torch.cat([i[1] for i in batch])
    
    label = label.squeeze(3)
    
    batch = Batch.from_data_list([b[0] for b in batch])
    
    return batch, label

train_dataset = datasetMaker(data_dict, train_i, edge_index, edge_attr, seq_len, future_steps, batch_size)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, drop_last=True, collate_fn=custom_collate)

val_dataset = datasetMaker(data_dict, val_i, edge_index, edge_attr, seq_len, future_steps, batch_size)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=True, drop_last=True, collate_fn=custom_collate)

test_dataset = datasetMaker(data_dict, test_i, edge_index, edge_attr, seq_len, future_steps, batch_size)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True, drop_last=True, collate_fn=custom_collate)


print("train len ", len(train_loader))
print("val len   ", len(val_loader))
print("test len  ", len(test_loader))


for data, label in train_loader:
    print(data)
    print(label.shape)
    break



In [ ]:

def reshape_to_batches(x, batch_description):
    """
        Does something like this:
        torch.Size([28, 576, 64]) --> torch.Size([4, 7, 576, 64])
    """
    num_splits = batch_description.max().item() + 1
    new_shape_dim_0 = num_splits
    new_shape_dim_1 = x.size(0) // new_shape_dim_0
    new_shape = torch.Size([new_shape_dim_0, new_shape_dim_1] + list(x.size()[1:]))
    reshaped_tensor = x.view(new_shape)
    return reshaped_tensor


class GCN(torch.nn.Module):
    def __init__(self, in_channels=1, gcn_hidden_channels=8, gcn_layers=1):
        super(GCN, self).__init__()
        self.in_conv = GCNConv(in_channels, gcn_hidden_channels)
        self.hidden_convs = [GCNConv(gcn_hidden_channels, gcn_hidden_channels).cuda() for i in range(gcn_layers - 1)]

    def forward(self, x, edge_index, batch):
        x = x.float()
        x = self.in_conv(x, edge_index)
        for conv in self.hidden_convs:
            x = F.relu(x)
            x = conv(x, edge_index)
        x = F.relu(x)
        return x

class MultiStepLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(MultiStepLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        
        # Flatten LSTM parameters
        self.lstm.flatten_parameters()
        
        # Fully connected layer to map LSTM output to desired output_size
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x, future=1):
        
        predictions = []
        
        
        for _ in range(future):
            # Initialize hidden state and cell state        
            batch_size, sequence_length, _ = x.size()
            h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)
            c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)

            # LSTM forward pass
            out, (h0, c0) = self.lstm(x, (h0, c0))
            
            pred = out[:, -1, :]
            
            x = torch.cat((x, pred.unsqueeze(1)), dim=1)
            
            t = self.fc(pred)
            predictions.append(t) # Append occupancy to predictions
            

        # Stack predictions along the sequence length dimension'
        predictions = torch.cat(predictions, dim=1)
        
        predictions = torch.unsqueeze(predictions, dim = 2)
        return predictions
    
class STGCN(nn.Module):
    
    def __init__(self, in_channels, gcn_layers, hidden_channels, lstm_layers, out_channels):
        super(STGCN, self).__init__()
        
        print("\033[100mhidden_channels:", hidden_channels,
              "   GCN hidden layers:", gcn_layers,
              "   lstm_layers:", lstm_layers, "\033[0m")

        
        self.GCN = GCN(in_channels=in_channels, gcn_hidden_channels=hidden_channels, gcn_layers=gcn_layers)

        self.lstm = MultiStepLSTM(hidden_channels, hidden_channels, out_channels, lstm_layers)
        
    def forward(self, data):    
 
        batch = data.batch
        
        data.x = data.x.float()  # Convert node features to Double
        data.edge_attr = data.edge_attr.float()  # Convert edge attributes to Double
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
       
        # Spatial processing
        x = self.GCN(x, edge_index, edge_attr)

        x = reshape_to_batches(x, batch)
        # Reshape and pass data through the model for each station
        predictions = []
        for station_data in x.permute(1,0,2,3):  # Iterate over each station
            #station_data = station_data.permute(1, 0, 2)  # Reshape for LSTM (batch_first=True)
            output = self.lstm(station_data, future=future_steps)
            predictions.append(output)

        # Concatenate predictions for all stations
        predictions = torch.stack(predictions, dim=1)
        return predictions

# Example usage:
# Define the adjacency matrix for spatial processing (A_spatial)
# Define the input size, number of layers, and number of heads for the temporal transformer


# Training

In [ ]:


def train_epoch(epoch, optimizer, loss_function, model, train_loader):
    total_loss = 0
    model.train()
    for batch_idx, (data,label) in enumerate(train_loader):
        #if batch_idx % 200== 0:
        #    print(str(batch_idx) + "/" + str(len(train_loader)), " ", total_loss)
            
        #    break

        
        label = reshape_to_batches(label, data.batch)
        data = data.cuda()
        label = label.cuda().float()
                
        optimizer.zero_grad()
        
        predictions = model(data)
                
        loss_value = loss_function(predictions,label)
        loss_value.backward()
        optimizer.step()

        total_loss += loss_value.item()
    return total_loss / len(train_loader)

def validate_epoch(epoch, loss, model, val_loader):
    total_loss = 0
    model.eval()

    with torch.no_grad():
        for batch_idx, (data, label) in enumerate(val_loader):
            #if batch_idx % 200 == 0:
            #    print(str(batch_idx) + "/" + str(len(val_loader)), " ", total_loss)
         
            label = reshape_to_batches(label, data.batch)
            data = data.cuda()
            label = label.cuda().float()
            
            predictions = model(data)
            
            loss_value = loss(predictions, label)
            total_loss += loss_value.item()
    return total_loss / len(val_loader)

def a_proper_training(num_epoch, model, optimizer, loss_function, train_loader, val_loader, scheduler):
    best_epoch = None
    best_model = None
    best_loss = None
    train_losses = list()
    val_losses = list()
    print("Begin Training")

    for epoch in range(num_epoch):
        start_time = time.time()  # Start time
        train_loss = train_epoch(epoch, optimizer, loss_function, model, train_loader)
        val_loss = validate_epoch(0, criterion, model, val_loader)
        train_losses.append(train_loss)
        val_losses.append(val_loss)

        scheduler.step(val_loss)  # Update the learning rate based on the validation loss

        if epoch == 0 or val_loss < best_loss:
            best_loss = val_loss
            best_model = copy.deepcopy(model)
            best_epoch = epoch

        end_time = time.time()
        elapsed_time = end_time - start_time
        
        print(f"Epoch {epoch + 1}/{num_epoch}: Train Loss = {train_loss} Val Loss = {val_loss} Elapsed_time = {elapsed_time // 60}mins")

    return (best_model, best_epoch, train_losses, val_losses)


In [ ]:
if True:
    model = STGCN(in_channels=1, gcn_layers=1, hidden_channels=8, lstm_layers=1, out_channels=1).cuda()

    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.MSELoss()

    scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=5, verbose=True)

    # Now pass the scheduler to the training function
    best_model, best_epoch, train_losses, val_losses = a_proper_training(
        30, model, optimizer, criterion, train_loader, val_loader, scheduler
    )

    torch.save(best_model.state_dict(), "best_ST-GCN_model_direct-connect.pth")

In [ ]:
plt.plot(train_losses, label="train")
plt.plot(val_losses, label="val")
plt.title("MSE Loss")
plt.legend()

In [ ]:
import optuna
from optuna.trial import TrialState
import torch.optim as optim
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
import copy
import time


NUM_EPOCHS = 250

def objective(trial):
    print("\033[41m-------------------------------------------------------------------------------------\033[0m")
    try:
        # Suggest hyperparameters with even values
        hidden_channels = trial.suggest_int('hidden_channels', 4, 20, step=2)
        gcn_layers = trial.suggest_int('gcn_layers', 1, 4)
        lstm_layers = trial.suggest_int('lstm_layers', 1, 10)
        learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True) 

        model = STGCN(in_channels=1, gcn_layers=gcn_layers, hidden_channels=hidden_channels, lstm_layers=lstm_layers, out_channels=1).cuda()

        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        criterion = nn.MSELoss()
        scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=5, verbose=True)

        best_loss = float('inf')
        patience = 10  # Number of epochs to wait for improvement before stopping
        patience_counter = 0  # Counter for epochs without improvement        
        best_model = None
        train_losses = list()
        val_losses = list()
        
        for epoch in range(NUM_EPOCHS):
            train_loss = train_epoch(epoch, optimizer, criterion, model, train_loader)
            val_loss = validate_epoch(epoch, criterion, model, val_loader)
            train_losses.append(train_loss)
            val_losses.append(val_loss)

            scheduler.step(val_loss)

            if val_loss < best_loss:
                best_loss = val_loss
                best_model = copy.deepcopy(model)
                patience_counter = 0  # Reset counter if improvement is observed
            else:
                patience_counter += 1  # Increment counter if no improvement

            if patience_counter >= patience:
                print(f"\033[34mStopping early at epoch {epoch} due to no improvement in validation loss.\033[0m")
                break  # Exit the loop if the model hasn't improved for 'patience' epochs
        
        plt.plot(train_losses, label="train")
        plt.plot(val_losses, label="val")
        plt.title("MSE Loss, lr=" + str(learning_rate))
        plt.legend()
        plt.savefig(f'models/direct_connect_lstm/model_{hidden_channels}{gcn_layers}{lstm_layers}_{str(best_loss)[2:8]}.png')
        torch.save(best_model.state_dict(), f'models/direct_connect_lstm/model_{hidden_channels}{gcn_layers}{lstm_layers}_{str(best_loss)[2:8]}.pth')

        print()
        return best_loss
    except Exception as e:
        print(e)
        return float('inf')

# Optimize hyperparameters
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)  # Define the number of trials

print("Best trial:")
trial = study.best_trial

print(" Value: ", trial.value)
print(" Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")
